<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [14]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/91.0.4472.124 Safari/537.36"
}


#headers = {
#"User-Agent": "DataScienceProject_falcon9_assignment/1.0 (alig@live.ie) Python/Requests"
#}


In [19]:
import requests

OLDID = "1027686922"

API_URL = "https://en.wikipedia.org/w/api.php"

params = {
    "action": "parse",      # Parse wikitext into HTML
    "oldid": OLDID,         # Specific revision ID
    "format": "json",
    "prop": "text"          # We want the rendered HTML content
}

headers = {
    "User-Agent": "MyFalconResearchBot/1.0 (contact: alig95@live.ie)"
}

response = requests.get(API_URL, params=params, headers=headers)
response.raise_for_status()

data = response.json()

# Extract rendered HTML
html = data["parse"]["text"]["*"]

# Save to a file
with open("falcon_launches_revision.html", "w", encoding="utf-8") as f:
    f.write(html)

print("Saved HTML of the revision to falcon_launches_revision.html")


Saved HTML of the revision to falcon_launches_revision.html


Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [15]:
# use requests.get() method with the provided static_url and headers
# assign the response to a object
response=requests.get(static_url)

Create a `BeautifulSoup` object from the HTML `response`


In [16]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

soup = BeautifulSoup(response.text, 'html.parser')


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [17]:
# Use soup.title attribute
print(soup.title)

None


In [18]:
print(response.text[:500])


Please set a user-agent and respect our robot policy https://w.wiki/4wJS. See also T400119.



In [21]:
import requests
from bs4 import BeautifulSoup

# --- 1. Wikipedia static revision URL ---
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# --- 2. IMPORTANT: Use a descriptive User-Agent (required by Wikipedia bot policy) ---
headers = {
    "User-Agent": "MyFalconResearchBot/1.0 (contact: your_email@example.com)"
}

# --- 3. Send request with headers ---
response = requests.get(static_url, headers=headers)

print("Status code:", response.status_code)

# --- 4. Proceed only if page loaded successfully ---
if response.status_code == 200:

    # --- 5. Create BeautifulSoup parser from the response ---
    soup = BeautifulSoup(response.text, "html.parser")

    # --- 6. Print <title> tag ---
    print("Page title:", soup.title)

    # --- 7. Print first 500 characters of retrieved HTML ---
    print("\nFirst 500 characters of page:")
    print("-------------------------------------")
    print(response.text[:500])
    print("-------------------------------------\n")

    # --- 8. Save full HTML to file ---
    with open("falcon_static_page.html", "w", encoding="utf-8") as f:
        f.write(response.text)

    print("Saved full HTML to: falcon_static_page.html")

else:
    print("Request failed:")
    print(response.text)


Status code: 200
Page title: <title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

First 500 characters of page:
-------------------------------------
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vect
-------------------------------------

Saved full HTML to: falcon_static_page.html


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [22]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Find all tables in the HTML
html_tables = soup.find_all("table")

print(f"Found {len(html_tables)} tables on the page.")

Found 25 tables on the page.


Starting from the third table is our target table contains the actual launch records.


In [23]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [26]:
column_names = []

# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

# Find all <th> header cells inside the first table
header_cells = first_launch_table.find_all("th")

column_names = []

for th in header_cells:
    name = extract_column_from_header(th)   # Apply the helper function
    if name is not None and len(name) > 0:  # Only keep non-empty column names
        column_names.append(name)

print("Extracted column names:")
print(column_names)



Extracted column names:
['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [27]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [28]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [32]:
# --- Safe text helper ---
def safe_text(cell):
    """Safely extract text from a BeautifulSoup table cell."""
    if cell is None:
        return ""
    return cell.get_text(strip=True) or ""

# --- Extraction loop ---
extracted_row = 0

for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):

        # Check if row has a flight number
        if rows.th and rows.th.string:
            flight_number = rows.th.string.strip()
            flag = flight_number.isdigit()
        else:
            flag = False

        # Extract all cells in the row
        row = rows.find_all('td')

        # Process only if this row corresponds to a launch
        if flag and len(row) >= 9:  # ensure there are enough columns
            extracted_row += 1

            # Date and time
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            time = datatimelist[1]

            # Booster version
            bv = booster_version(row[1])
            if not bv:
                bv = safe_text(row[1])
            print("Booster Version:", bv)

            # Launch site
            launch_site = safe_text(row[2])

            # Payload
            payload = safe_text(row[3])

            # Payload mass
            payload_mass = get_mass(row[4])

            # Orbit
            orbit = safe_text(row[5])

            # Customer
            customer = safe_text(row[6])

            # Launch outcome
            launch_outcome = safe_text(row[7])

            # Booster landing
            booster_landing = landing_status(row[8])

            # Optional: Print the full extracted row
            print(f"{flight_number} | {date} {time} | {bv} | {launch_site} | {payload} | {payload_mass} | {orbit} | {customer} | {launch_outcome} | {booster_landing}")


Booster Version: F9 v1.07B0003.18
1 | 4 June 2010 18:45 | F9 v1.07B0003.18 | CCAFS,SLC-40 | Dragon Spacecraft Qualification Unit | 0 | LEO | SpaceX | Success | Failure
Booster Version: F9 v1.07B0004.18
2 | 8 December 2010 15:43 | F9 v1.07B0004.18 | CCAFS,SLC-40 | Dragondemo flight C1(Dragon C101) | 0 | LEO(ISS) | NASA(COTS)NRO | Success[9] | Failure
Booster Version: F9 v1.07B0005.18
3 | 22 May 2012 07:44 | F9 v1.07B0005.18 | CCAFS,SLC-40 | Dragondemo flight C2+[18](Dragon C102) | 525 kg | LEO(ISS) | NASA(COTS) | Success[20] | No attempt

Booster Version: F9 v1.07B0006.18
4 | 8 October 2012 00:35 | F9 v1.07B0006.18 | CCAFS,SLC-40 | SpaceX CRS-1[22](Dragon C103) | 4,700 kg | LEO(ISS) | NASA(CRS) | Success | No attempt
Booster Version: F9 v1.07B0007.18
5 | 1 March 2013 15:10 | F9 v1.07B0007.18 | CCAFS,SLC-40 | SpaceX CRS-2[22](Dragon C104) | 4,877 kg | LEO(ISS) | NASA(CRS) | Success | No attempt

Booster Version: F9 v1.17B10038
6 | 29 September 2013 16:00 | F9 v1.17B10038 | VAFB,SLC-4E | 

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [33]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
